In this script I use the Amazon Featured Offer Demand system as the environment. We obtained this demand system using the high-frequency and large-scale data collected from the top 5000 best seller ASINs in the "Books" section from Amazon (by Nov. 2024). We assume that the Demand system is still in logit form, but the winner of the Buy Box can obtain an additional utility in the demand that makes it significantly larger compared with other counterparts. 

The Amazon Buy Box assignment system is built upon predictions from absorbed results.